# 02 · Single GMRES run

Drive the GMRES solver for a single configuration and track residuals.

### Load the essentials

In [25]:
# %% Setup (hot-reload while you tweak src/*)
%load_ext autoreload
%autoreload 2

# Core API from your package
from src import GridSpec, GMRESOptions, gmres_solve

# Operators / loads / viz helpers
from src.operators import helmholtz_operator, BC
from src.loads import PointSource
from src.visualisation import plot_residuals, plot_field


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Versions

Useful when comparing runs across environments (Python / SciPy / package).


In [26]:
# %% Version info
import sys, numpy as np
try:
    import scipy
except Exception:
    scipy = None

try:
    import src as pkg
    pkg_version = getattr(pkg, "__version__", "unknown")
except Exception:
    pkg_version = "unknown"

print("Python :", sys.version.split()[0])
print("NumPy  :", np.__version__)
print("SciPy  :", getattr(scipy, "__version__", "not installed"))
print("Package:", pkg_version)


Python : 3.12.6
NumPy  : 2.3.3
SciPy  : 1.16.2
Package: 0+local


## Parameters

Tweak everything here: grid size, wavenumber `k`, boundary condition `bc`, dtype, and GMRES options.


In [27]:
# %% Parameters
# Grid
shape   = (60, 60)
lengths = (1.0, 1.0)

# Helmholtz
k     = 30.0                 # wavenumber |k|
bc    = BC.DIRICHLET         # try: BC.NEUMANN, BC.PERIODIC
dtype = "complex128"         # "float64" if you want a purely real system

# Right-hand side (point source)
source_location = "centre"   # or (x, y) in physical coords, or "random"
source_amp      = 1.0

# GMRES options
gmres_opts = GMRESOptions(tol=1e-6, maxiter=200, restart=None)


## Build grid


In [28]:
# %% Grid
grid = GridSpec(dims=2, shape=shape, lengths=lengths)
grid


GridSpec(dims=2, shape=(60, 60), lengths=(1.0, 1.0))

## Assemble Helmholtz operator

Build \(A = \Delta + k^2 I\) with the selected boundary condition.


In [29]:
# %% Operator
A = helmholtz_operator(grid, k=k, bc=bc, dtype=dtype)
A.shape, A.nnz


((3600, 3600), 17056)

## Build the RHS (point source)

`PointSource` understands keywords (`"centre"`, `"origin"`, `"random"`) or physical coordinates.


In [30]:
# %% RHS
load = PointSource(location=source_location, amplitude=source_amp)
b = load.build(grid)
b.shape, (b != 0).sum()    # show size and number of non-zeros (should be 1)


((3600,), np.int64(1))

VANAF HIER ERROR

## Run GMRES

Solve \(A u = b\) and report convergence.  
Residual history is recorded through the options-powered callback in `gmres_solve`.


In [31]:
# %% Solve (GMRES)
result = gmres_solve(A, b, options=gmres_opts)

print(f"Converged: {result.converged} | info: {result.info}")
if result.residuals:
    print(f"Iterations: {len(result.residuals)} | Final residual: {result.residuals[-1]:.2e}")


TypeError: gmres() got an unexpected keyword argument 'tol'

## Diagnostics

Plot residual history and the real part of the solution field.


In [33]:
# %% Diagnostics (residuals)
ax = plot_residuals(result)
ax.figure.tight_layout() if ax is not None else None


NameError: name 'result' is not defined

### Next steps

- Try `bc = BC.NEUMANN` or `BC.PERIODIC` and compare convergence.
- Sweep over `k` (e.g., 10, 20, 30, 40) to see how it affects GMRES.
- Replace `PointSource` with `PlaneWaveSource` or `RandomSource` for different RHS types.
- Switch `dtype` to `"float64"` if your loads/solutions are real-valued.
